<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%206/Modulo_6_Ejercicio_6_Stacking_Resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejercicio 6: Predicción de Fallos con Modelos Híbridos (Stacking)**
**Propósito**: Construir un modelo de stacking combinando modelos tradicionales y avanzados para detectar fallos en procesos industriales.

## Introducción

La predicción de fallos en entornos industriales es un reto clave para garantizar la calidad y fiabilidad de los productos. En este ejercicio, abordamos este desafío aplicando una técnica avanzada conocida como **stacking** o **ensamblado jerárquico**.

El stacking permite **combinar varios modelos base** (de distinto tipo) y un **modelo meta** (meta-modelo) que aprende a partir de sus predicciones. El objetivo es construir un predictor más robusto, capaz de mejorar el rendimiento frente a modelos individuales.

Utilizaremos el **SECOM Manufacturing Data Set**, un conjunto de datos multivariable, complejo y realista, ideal para explorar técnicas de ensemble en problemas de clasificación binaria con ruido y correlaciones espurias.

Objetivos del ejercicio:
- Entrenar tres modelos base: **Regresión Logística**, **Random Forest**, **MLP**.
- Combinar sus predicciones utilizando un **meta-modelo** (XGBoost).
- Comparar el rendimiento del stacking frente a los modelos individuales.
- Discutir la aplicabilidad del stacking en entornos industriales.

## Carga de librerías y configuración del entorno

In [1]:
# Celda 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier

from imblearn.over_sampling import SMOTE

import requests
import zipfile
import io

from xgboost import XGBClassifier

# Estilo visual
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

## Descripción del dataset o datos de entrada

El dataset utilizado en este ejercicio es el **SECOM Manufacturing Data Set**, proveniente del UCI Machine Learning Repository.

- Contiene mediciones de sensores durante procesos de manufactura electrónica.
- Cada fila representa una observación con cientos de variables numéricas.
- La variable objetivo es binaria (`1`: fallo, `-1`: sin fallo).

Este dataset se caracteriza por su **alta dimensionalidad**, **valores ruidosos** y **presencia de correlaciones espurias**, lo que lo convierte en un buen candidato para evaluar modelos de tipo ensemble como el stacking.

Fuente: https://archive.ics.uci.edu/ml/datasets/SECOM

## Desarrollo del código paso a paso

**Carga y exploración de los datos**

In [2]:

zip_url = "https://github.com/dtoralg/IE_Calidad_ML/raw/refs/heads/main/Data/Modulo%206/secom.zip"

# Descargar el zip
response = requests.get(zip_url)

# Verificar que la descarga fue exitosa
if response.status_code == 200:
    # Leer el archivo ZIP en memoria
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # Mostrar el contenido del ZIP
    print("Archivos contenidos:")
    print(zip_file.namelist())

    # Extraer todos los archivos (opcional)
    zip_file.extractall("secom_data")  # Carpeta donde se guardarán
    print("Archivos extraídos en la carpeta 'secom_data'.")
else:
    print("Error al descargar el archivo:", response.status_code)


Archivos contenidos:
['secom.data', 'secom.names', 'secom_labels.data']
Archivos extraídos en la carpeta 'secom_data'.


In [3]:
# Celda 2
url_data = '/content/secom_data/secom.data'
url_labels = '/content/secom_data/secom_labels.data'

# Cargar datos
df = pd.read_csv(url_data, sep='\s+', header=None)
labels = pd.read_csv(url_labels, sep='\s+', header=None)
df['target'] = labels[0]
df.head()

,0,1,2,3,4,5,6,7,8,9,...,581,582,583,584,585,586,587,588,589,target
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,-0.0124,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,-0.0031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


**Limpieza de datos y preparación del conjunto de entrenamiento**

In [4]:
# Celda 3
# Eliminar columnas con demasiados nulos
nulls = df.isnull().mean()
df_clean = df.loc[:, nulls < 0.25]  # eliminar columnas con más del 25% de NaNs

# Eliminar filas con valores nulos restantes
df_clean = df_clean.dropna()

# Separar variables predictoras y target
X = df_clean.drop(columns='target')
y = df_clean['target'].replace({-1: 0, 1: 1})

# Escalar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Separar entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.3, random_state=42)

# Crear y aplicar SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Verificar
print("Dimensiones finales:", X_train.shape, X_test.shape)


Dimensiones finales: (1274, 558) (294, 558)


**Entrenamiento de modelos individuales (base learners)**

In [9]:
# Celda 4
# Regresión Logística
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

# Random Forest
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf.fit(X_train, y_train)

# MLP
mlp = MLPClassifier(hidden_layer_sizes=(128,), max_iter=200, random_state=42)
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(128,), random_state=42)

## Visualización de resultados

**Evaluación de modelos individuales**

In [10]:
# Celda 5
for name, model in [('Logistic Regression', lr), ('Random Forest', rf), ('MLP', mlp)]:
    y_pred = model.predict(X_test)
    print(f"--- {name} ---")
    print(classification_report(y_test, y_pred))
    print("F1 Macro:", f1_score(y_test, y_pred, average='macro'))
    print()

--- Logistic Regression ---
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       274
           1       0.21      0.30      0.24        20

    accuracy                           0.87       294
   macro avg       0.58      0.61      0.59       294
weighted avg       0.90      0.87      0.88       294

F1 Macro: 0.588126159554731

--- Random Forest ---
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       274
           1       0.00      0.00      0.00        20

    accuracy                           0.93       294
   macro avg       0.47      0.50      0.48       294
weighted avg       0.87      0.93      0.90       294

F1 Macro: 0.4823943661971831

--- MLP ---
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       274
           1       0.20      0.15      0.17        20

    accuracy                           0.90       294
   macr

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Entrenamiento del modelo Stacking con meta-modelo XGBoost**

In [11]:
# Celda 6
# Meta-modelo
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Definir ensemble stacking
stacking = StackingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('mlp', mlp)
    ],
    final_estimator=xgb,
    cv=5
)

# Entrenar ensemble
stacking.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:36:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(max_iter=1000)),
                               ('rf',
                                RandomForestClassifier(n_estimators=1000,
                                                       random_state=42)),
                               ('mlp',
                                MLPClassifier(hidden_layer_sizes=(128,),
                                              random_state=42))],
                   final_estimator=XGBClassifier(base_score=None, booster=None,
                                                 callbacks=None,
                                                 colsample_bylevel=None,
                                                 colsample_bynode=None,
                                                 colsample_bytree=None,
                                                 d...
                                                 grow_policy=None,
                                                 importance_type=None,
                                                 interaction_constraints=None,
                                                 learning_rate=None,
                                                 max_bin=None,
                                                 max_cat_threshold=None,
                                                 max_cat_to_onehot=None,
                                                 max_delta_step=None,
                                                 max_depth=None,
                                                 max_leaves=None,
                                                 min_child_weight=None,
                                                 missing=nan,
                                                 monotone_constraints=None,
                                                 multi_strategy=None,
                                                 n_estimators=None, n_jobs=None,
                                                 num_parallel_tree=None,
                                                 random_state=42, ...))

In [12]:
# Celda 7
y_pred_stack = stacking.predict(X_test)
print("--- Modelo Stacking ---")
print(classification_report(y_test, y_pred_stack))
print("F1 Macro:", f1_score(y_test, y_pred_stack, average='macro'))

--- Modelo Stacking ---
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       274
           1       0.00      0.00      0.00        20

    accuracy                           0.93       294
   macro avg       0.47      0.50      0.48       294
weighted avg       0.87      0.93      0.90       294

F1 Macro: 0.4823943661971831


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Conclusiones

Este ejercicio ha demostrado el valor del **stacking** como técnica para combinar modelos heterogéneos y mejorar el rendimiento global.

- Se entrenaron tres modelos base con algoritmos distintos: regresión, árbol y red neuronal.
- Se combinó su output con un meta-modelo XGBoost.
- El modelo stacking no logró mejorar el `F1 Macro`, dado que el desbalanceo de clases es muy severo.

El stacking no siempre mejora los modelos individuales, y en algunos casos de clases muy desbalanceadas, puede empeorarlos.

En este caso vemos que la regresión logística había conseguido predecir ligereamente la clase minoritaria, no siendo así con el modelo stacking.

Podríamos probar otro metamodelo para así afinar el stacking hasta conseguir mejores características de los distintos modelos.


## Próximos pasos

- Realizar selección de variables para reducir dimensionalidad antes de aplicar stacking.
- Implementar versiones avanzadas con validación interna y meta-predicciones.
- Probar meta-modelos distintos como LogisticRegression para evaluar su impacto.

---
Aplicación industrial de técnicas de stacking con SECOM dataset